# Comparação entre treinos; uso de dropouts e mais camadas ocultas


In [39]:
import seaborn as sns
import pandas as pd

obtensão dos dados

In [62]:
titanic_df = sns.load_dataset('titanic')
titanic_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


separar os dados que nos são relevantes

In [63]:
feature_names = ['pclass', 'female', 'age', 'fare'] # classe, feminino, idade, preço da passagem
titanic_df['female'] = titanic_df['sex'].map({'male': 0, 'female':1})
titanic_df['alive'] = titanic_df['alive'].map({'no': 0, 'yes':1})
titanic_df.dropna(subset=feature_names, inplace=True)

titanic_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,female
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,0,False,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,1,False,1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,1,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,1,False,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,0,True,0


Transformar em matrizes do numpy

In [64]:
X = titanic_df[feature_names].to_numpy()
Y = titanic_df['alive'].to_numpy()

Separar em treino e teste

In [85]:
#!pip install scikit-learn
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.2,random_state=42,stratify=Y) 

##### As classes a serem comparadas

In [45]:
#!pip install torch  --index-url https://download.pytorch.org/whl/cpu

import torch
import torch.nn as nn
import torch.nn.functional as F

In [144]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.fc1 = nn.Linear(4, 20) # primeira hidden layer
        self.fc2 = nn.Linear(20, 1) # segunda hidden layer
        self.sigmoid = nn.Sigmoid() # output layer com ativação Sigmoid

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

modelo_simples = ClassBin()

In [167]:
class ClassBin2(nn.Module):
    def __init__(self):
        super(ClassBin2, self).__init__()
        self.fc1 = nn.Linear(4, 16)  
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(16, 8) 
        self.dropout2 = nn.Dropout(0.2) 
        self.fc3 = nn.Linear(8,1)   
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x) 
        return x
    
modelo_rebuscado = ClassBin2()

## Modelo Simples

Preparando variaveis para sessão de treino

In [146]:
loos_fn = nn.BCELoss()
max_epochs = 100
batch_size = 32
learning_rate = 0.001 # começando alto ne, diminui automático será ?

optimizer = torch.optim.Adam(modelo_simples.parameters(), lr=learning_rate)

Convertendo dados para tensores

In [147]:
from torch.utils.data import DataLoader, TensorDataset

# numpy -> torch.Tensor
X_train = torch.Tensor(x_train)
X_test = torch.Tensor(x_test)
Y_train = torch.Tensor(y_train)
Y_test = torch.Tensor(y_test)

# torch usa objetos de DataSets pra facilitar fazer os batchs 
train = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test, batch_size=batch_size)

Treinamento 

In [148]:
def avalia(model):
    model.eval()
    
    with torch.no_grad():  # Desabilitar gradientes durante avaliação
        train_pred = model(X_train)
        train_pred_binary = (train_pred > 0.5).float().flatten()
        train_acc = torch.sum(train_pred_binary == Y_train) / len(Y_train)

        test_pred = model(X_test)
        test_pred_binary = (test_pred > 0.5).float().flatten()
        test_acc = torch.sum(test_pred_binary == Y_test) / len(Y_test)

    return train_acc, test_acc

In [149]:
modelo_simples.train()

for epoch in range(max_epochs):

    for batch in train_loader:

        x = batch[0]
        y = batch[1]

        y_pred = modelo_simples(x) # chama forward internamente e mais um pá de coisas, pelo dunder __call__

        # Usar unsqueeze sem modificar in-place para evitar problemas
        loss = loos_fn(y_pred, y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    if(epoch % 10 == 0):
        print(f"epoca {epoch}: loss = {loss.item():.3f}")

epoca 0: loss = 0.841
epoca 10: loss = 0.511
epoca 20: loss = 0.580
epoca 30: loss = 0.451
epoca 40: loss = 0.638
epoca 50: loss = 0.552
epoca 60: loss = 0.489
epoca 70: loss = 0.515
epoca 80: loss = 0.516
epoca 90: loss = 0.583


Avaliacao

In [150]:
s_train_acc, s_test_acc = avalia(modelo_simples)

## Modelo Rebuscado

In [ ]:
loos_fn = nn.BCELoss()
max_epochs = 100
batch_size = 32
learning_rate = 0.001 # começando alto ne, diminui automático será ?

optimizer = torch.optim.Adam(modelo_rebuscado.parameters(), lr=learning_rate)

In [177]:
from torch.utils.data import DataLoader, TensorDataset

# numpy -> torch.Tensor
X_train = torch.Tensor(x_train)
X_test = torch.Tensor(x_test)
Y_train = torch.Tensor(y_train)
Y_test = torch.Tensor(y_test)

# torch usa objetos de DataSets pra facilitar fazer os batchs 
train = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test, batch_size=batch_size)

In [178]:
modelo_rebuscado.train()

for epoch in range(max_epochs):

    for batch in train_loader:

        x = batch[0]
        y = batch[1]

        y_pred = modelo_rebuscado(x) # chama forward internamente e mais um pá de coisas, pelo dunder __call__

        # Usar unsqueeze sem modificar in-place para evitar problemas
        loss = loos_fn(y_pred, y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    if(epoch % 10 == 0):
        print(f"epoca {epoch}: loss = {loss.item():.3f}")

epoca 0: loss = 0.454
epoca 10: loss = 0.454
epoca 20: loss = 0.551
epoca 30: loss = 0.524
epoca 40: loss = 0.667
epoca 50: loss = 0.539
epoca 60: loss = 0.467
epoca 70: loss = 0.390
epoca 80: loss = 0.356
epoca 90: loss = 0.507
epoca 100: loss = 0.380
epoca 110: loss = 0.286
epoca 120: loss = 0.564
epoca 130: loss = 0.545
epoca 140: loss = 0.667
epoca 150: loss = 0.544
epoca 160: loss = 0.551
epoca 170: loss = 0.389
epoca 180: loss = 0.446
epoca 190: loss = 0.701


In [179]:
r_train_acc, r_test_acc = avalia(modelo_rebuscado)

## Comparação de modelos

In [180]:
print('\n-------------- SIMPLES --------------\n')
print(f"Acurácia de Treino: {s_train_acc:.4f}")
print(f"Acurácia de Teste: {s_test_acc:.4f}")

print('\n-------------- REBUSCADO --------------\n')
print(f"Acurácia de Treino: {r_train_acc:.4f}")
print(f"Acurácia de Teste: {r_test_acc:.4f}")


-------------- SIMPLES --------------

Acurácia de Treino: 0.7846
Acurácia de Teste: 0.7483

-------------- REBUSCADO --------------

Acurácia de Treino: 0.8021
Acurácia de Teste: 0.8042
